# Ноутбук администратора для загрузки "красных флажков"
Для работы по загрузке "`красных флажков`" первоначально необходимо создать коллекцию в векторном хранилище `Qdrant`. Все действия производятся с рабочего места администратора, поэтому первоначально необходимо развернуть виртуальное окружение.

Данные действия необходимо сделать в терминале Linux вне данного ноутбука
```bash
    python3 -m venv .venv
    source .venv/bin/activate
```

## Порядок работы
1. Юрист заполняет CSV-файл `data/red_flags_ru.csv` по шаблону (колонки: `id`, `document_type`, `document_type_ru`, `category`, `severity`, `flag_text`, `lang`)
2. Администратор запускает данный ноутбук для векторизации и загрузки в Qdrant
3. Ноутбук автоматически создаёт эмбеддинги и загружает их в коллекцию `ru_documents_embeddings`

## 1. Установка зависимостей и импорт библиотек

In [19]:
# установка библиотек
!pip install qdrant-client python-dotenv sentence-transformers pandas -q

import os
import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import HnswConfigDiff
from qdrant_client.models import (
    VectorParams, Distance, PointStruct, 
    Filter, FieldCondition, MatchValue
)
from sentence_transformers import SentenceTransformer

## 2. Загрузка переменных окружения и подключение к Qdrant
API-ключ загружается из файла `configuration/.env`. Путь указан относительно корня проекта.

In [7]:
# Загрузка переменных окружения из configuration/.env
ENV_PATH = os.path.join("..", "..", "configuration", ".env")

if not os.path.exists(ENV_PATH):
    raise FileNotFoundError(
        f"Файл .env не найден по пути: {os.path.abspath(ENV_PATH)}\n"
        f"Убедитесь, что файл configuration/.env существует."
    )

load_dotenv(ENV_PATH)

QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
if not QDRANT_API_KEY:
    raise ValueError("QDRANT_API_KEY не задан в файле configuration/.env")

print("✅ Переменные окружения загружены успешно")

✅ Переменные окружения загружены успешно


In [8]:
# Подключение к серверу Qdrant
qdrant_url = input("Введите URL сервера Qdrant (например, http://localhost): ")
qdrant_port = int(input("Введите порт Qdrant (по умолчанию 6333): ") or "6333")

client = QdrantClient(
    url=f"{qdrant_url}:{qdrant_port}",
    api_key=QDRANT_API_KEY,
)

# Проверка подключения
collections = client.get_collections()
print(f"✅ Подключение к Qdrant установлено. Существующие коллекции: {[c.name for c in collections.collections]}")

✅ Подключение к Qdrant установлено. Существующие коллекции: ['mayak_docs', 'red_flags', 'langflow2', 'federle', 'ru_documents_embeddings']


/tmp/ipykernel_73489/2480837799.py:5: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


## 3. Инициализация модели эмбеддингов
Используется модель `cointegrated/rubert-tiny2` (размерность вектора: 312) или `ai-forever/sbert_large_nlu_ru` (размерность: 1024).

По умолчанию выбрана модель `intfloat/multilingual-e5-base` (768 измерений), которая хорошо работает с русским текстом и соответствует размерности коллекции.

In [ ]:
# Модель загружается с Hugging Face Hub (https://huggingface.co/intfloat/multilingual-e5-base)
# В соответствии с требованием п.2 раздела "Сбор требований" (07_Database.md)
MODEL_NAME = "intfloat/multilingual-e5-base"  # 768 измерений, хорошая поддержка русского языка
VECTOR_SIZE = 768

print(f"⏳ Загрузка модели {MODEL_NAME}...")
model = SentenceTransformer(MODEL_NAME)
print(f"✅ Модель загружена. Размерность вектора: {model.get_sentence_embedding_dimension()}")

# Проверка размерности
assert model.get_sentence_embedding_dimension() == VECTOR_SIZE, \
    f"Размерность модели ({model.get_sentence_embedding_dimension()}) не совпадает с ожидаемой ({VECTOR_SIZE})"

⏳ Загрузка модели intfloat/multilingual-e5-base...
✅ Модель загружена. Размерность вектора: 768


## 4. Создание коллекции в Qdrant
Создаётся коллекция `ru_documents_embeddings` для хранения русскоязычных "красных флажков".

In [20]:
COLLECTION_NAME = "ru_documents_embeddings"

# Проверяем существование коллекции
existing = [c.name for c in client.get_collections().collections]

if COLLECTION_NAME in existing:
    action = input(f"Коллекция '{COLLECTION_NAME}' уже существует. Пересоздать? (y/n): ")
    if action.lower() == 'y':
        client.delete_collection(COLLECTION_NAME)
        print(f"🗑️ Коллекция '{COLLECTION_NAME}' удалена")
    else:
        print(f"ℹ️ Используется существующая коллекция '{COLLECTION_NAME}'")

if COLLECTION_NAME not in [c.name for c in client.get_collections().collections]:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=VECTOR_SIZE,
            distance=Distance.COSINE,
            on_disk=False  # хранение в RAM для скорости
        ),
        hnsw_config=HnswConfigDiff(
            m=16,
            ef_construct=200
        )
    )
    print(f"✅ Коллекция '{COLLECTION_NAME}' создана (размерность: {VECTOR_SIZE}, метрика: Cosine)")

🗑️ Коллекция 'ru_documents_embeddings' удалена
✅ Коллекция 'ru_documents_embeddings' создана (размерность: 768, метрика: Cosine)


## 5. Загрузка "красных флажков" из CSV
Шаблон CSV-файла расположен в `docs/red_flags_ru_templates.csv`. Юрист должен заполнить его по представленному ниже образцу. 
Непосредственно для импорта флажков в векторное хранилище CSV-файл должен находиться в `data/red_flags_ru.csv`.

Состав шаблона:

| Столбец | Описание | Пример |
|---------|----------|--------|
| `id` | Уникальный идентификатор | 1 |
| `document_type` | Тип документа (англ.) | user_agreement |
| `document_type_ru` | Тип документа (рус.) | Пользовательское соглашение |
| `category` | Категория риска | односторонние_изменения |
| `severity` | Уровень серьёзности | critical / high / medium |
| `flag_text` | Текст красного флажка | Сервис оставляет за собой... |
| `lang` | Язык | ru |

In [21]:
# Загрузка CSV с красными флажками
CSV_PATH = os.path.join("..", "..", "data", "red_flags_ru.csv")

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"CSV-файл не найден: {os.path.abspath(CSV_PATH)}\n"
        f"Поместите файл red_flags_ru.csv в директорию data/"
    )

df = pd.read_csv(CSV_PATH, encoding='utf-8')

# Валидация CSV
required_columns = ['id', 'document_type', 'document_type_ru', 'category', 'severity', 'flag_text', 'lang']
missing = set(required_columns) - set(df.columns)
if missing:
    raise ValueError(f"В CSV отсутствуют обязательные столбцы: {missing}")

# Фильтрация только русскоязычных флажков
df = df[df['lang'] == 'ru'].copy()

print(f"✅ Загружено {len(df)} красных флажков из CSV")
print(f"\nРаспределение по типам документов:")
print(df['document_type_ru'].value_counts().to_string())
print(f"\nРаспределение по уровню серьёзности:")
print(df['severity'].value_counts().to_string())

✅ Загружено 241 красных флажков из CSV

Распределение по типам документов:
document_type_ru
Оферта онлайн-школы / сервиса подписки    85
Политика конфиденциальности               79
Пользовательское соглашение               77

Распределение по уровню серьёзности:
severity
high        99
critical    83
medium      59


## 6. Генерация эмбеддингов и загрузка в Qdrant
Для модели `multilingual-e5-base` рекомендуется добавлять префикс `query: ` к текстам при создании эмбеддингов.

In [22]:
# Подготовка текстов для эмбеддинга
# Для multilingual-e5 рекомендуется добавлять префикс
texts = [f"query: {row['flag_text']}" for _, row in df.iterrows()]

print(f"⏳ Генерация эмбеддингов для {len(texts)} текстов...")
embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
print(f"✅ Эмбеддинги сгенерированы. Размерность: {embeddings.shape}")

⏳ Генерация эмбеддингов для 241 текстов...


Batches: 100%|██████████| 8/8 [00:25<00:00,  3.20s/it]

✅ Эмбеддинги сгенерированы. Размерность: (241, 768)


In [23]:
# Формирование точек для загрузки в Qdrant
BATCH_SIZE = 100

points = []
for idx, (_, row) in enumerate(df.iterrows()):
    point = PointStruct(
        id=int(row['id']),
        vector=embeddings[idx].tolist(),
        payload={
            "document_type": row['document_type'],
            "document_type_ru": row['document_type_ru'],
            "category": row['category'],
            "severity": row['severity'],
            "flag_text": row['flag_text'],
            "lang": row['lang']
        }
    )
    points.append(point)

# Загрузка батчами
total_uploaded = 0
for i in range(0, len(points), BATCH_SIZE):
    batch = points[i:i + BATCH_SIZE]
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=batch
    )
    total_uploaded += len(batch)
    print(f"  📦 Загружено {total_uploaded}/{len(points)} точек...")

print(f"\n✅ Все {total_uploaded} красных флажков загружены в коллекцию '{COLLECTION_NAME}'")

  📦 Загружено 100/241 точек...
  📦 Загружено 200/241 точек...
  📦 Загружено 241/241 точек...

✅ Все 241 красных флажков загружены в коллекцию 'ru_documents_embeddings'


## 7. Проверка загрузки
Тестовый поиск по коллекции для верификации корректности загрузки.

In [24]:
# Проверка количества записей в коллекции
collection_info = client.get_collection(COLLECTION_NAME)
print(f"📊 Статистика коллекции '{COLLECTION_NAME}':")
print(f"   Всего точек: {collection_info.points_count}")
print(f"   Размерность вектора: {collection_info.config.params.vectors.size}")
print(f"   Метрика: {collection_info.config.params.vectors.distance}")

📊 Статистика коллекции 'ru_documents_embeddings':
   Всего точек: 241
   Размерность вектора: 768
   Метрика: Cosine


In [25]:
# Тестовый поиск: проверяем релевантность
test_query = "query: Сервис может удалить аккаунт пользователя без предупреждения"
test_embedding = model.encode([test_query])[0].tolist()

results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=test_embedding,
    limit=5
)

print(f"🔍 Тестовый запрос: '{test_query}'\n")
print(f"Топ-5 результатов:")
for i, point in enumerate(results.points):
    print(f"  {i+1}. [{point.payload['severity']}] ({point.payload['document_type_ru']})")
    print(f"     {point.payload['flag_text']}")
    print(f"     Score: {point.score:.4f}")
    print()

🔍 Тестовый запрос: 'query: Сервис может удалить аккаунт пользователя без предупреждения'

Топ-5 результатов:
  1. [critical] (Пользовательское соглашение)
     Сервис вправе заблокировать или удалить аккаунт пользователя без объяснения причин
     Score: 0.9460

  2. [critical] (Пользовательское соглашение)
     Сервис может заблокировать аккаунт при подозрении на нарушение без предоставления доказательств
     Score: 0.9253

  3. [critical] (Пользовательское соглашение)
     При удалении аккаунта все данные пользователя удаляются безвозвратно, включая оплаченный контент
     Score: 0.9133

  4. [critical] (Политика конфиденциальности)
     Данные хранятся бессрочно даже после удаления аккаунта пользователя
     Score: 0.9025

  5. [critical] (Пользовательское соглашение)
     Сервис освобождается от ответственности за потерю данных пользователя при любых обстоятельствах
     Score: 0.9002



In [26]:
# Тестовый поиск с фильтрацией по типу документа
test_query_2 = "query: Невозможно вернуть деньги за курс"
test_embedding_2 = model.encode([test_query_2])[0].tolist()

results_filtered = client.query_points(
    collection_name=COLLECTION_NAME,
    query=test_embedding_2,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="document_type",
                match=MatchValue(value="subscription_offer")
            )
        ]
    ),
    limit=5
)

print(f"🔍 Тестовый запрос с фильтром (subscription_offer): '{test_query_2}'\n")
for i, point in enumerate(results_filtered.points):
    print(f"  {i+1}. [{point.payload['severity']}] {point.payload['category']}")
    print(f"     {point.payload['flag_text']}")
    print(f"     Score: {point.score:.4f}")
    print()

🔍 Тестовый запрос с фильтром (subscription_offer): 'query: Невозможно вернуть деньги за курс'

  1. [critical] оплата_возврат
     Возврат денежных средств за курс или подписку невозможен ни при каких обстоятельствах
     Score: 0.9541

  2. [medium] оплата_возврат
     Возврат невозможен для акционных и скидочных предложений
     Score: 0.9156

  3. [high] оплата_возврат
     Отсутствует чёткая процедура оформления возврата денежных средств
     Score: 0.9131

  4. [critical] маркетинг
     Гарантия возврата денег оговорена такими условиями, что воспользоваться ей практически невозможно
     Score: 0.9100

  5. [critical] обязанности_ученика
     При нарушении правил школы ученик теряет доступ к курсу без возврата оплаты
     Score: 0.9075



## 8. Создание индексов для фильтрации (опционально)
Создание payload-индексов ускоряет фильтрацию при поиске.

In [27]:
# Создание индексов для полей, по которым будет происходить фильтрация
from qdrant_client.models import PayloadSchemaType

for field in ["document_type", "severity", "category", "lang"]:
    client.create_payload_index(
        collection_name=COLLECTION_NAME,
        field_name=field,
        field_schema=PayloadSchemaType.KEYWORD
    )
    print(f"✅ Индекс создан для поля '{field}'")

print("\n🎉 Инициализация векторной базы данных завершена!")

✅ Индекс создан для поля 'document_type'
✅ Индекс создан для поля 'severity'
✅ Индекс создан для поля 'category'
✅ Индекс создан для поля 'lang'

🎉 Инициализация векторной базы данных завершена!
